In [1]:
import pandas as pd
import requests
import time

In [3]:
clean_22 = pd.read_pickle('../data_raw/df_filled_22.pkl')
clean_23 = pd.read_pickle('../data_raw/df_filled_23.pkl')
clean_22.head()

timestamp,2022-02-01 00:00:00,2022-02-01 01:00:00,2022-02-01 02:00:00,2022-02-01 03:00:00,2022-02-01 04:00:00,2022-02-01 05:00:00,2022-02-01 06:00:00,2022-02-01 07:00:00,2022-02-01 08:00:00,2022-02-01 09:00:00,...,2022-12-31 14:00:00,2022-12-31 15:00:00,2022-12-31 16:00:00,2022-12-31 17:00:00,2022-12-31 18:00:00,2022-12-31 19:00:00,2022-12-31 20:00:00,2022-12-31 21:00:00,2022-12-31 22:00:00,2022-12-31 23:00:00
station_complex_id,,,,,,,,,,,,,,,,,,,,,
1,17.0,5.0,6.0,12.0,44.0,143.0,405.0,735.0,950.0,512.0,...,426.0,402.0,304.0,338.0,322.0,350.0,309.0,293.0,168.0,75.0
10,100.0,24.0,7.0,8.0,8.0,14.0,43.0,98.0,175.0,182.0,...,3.0,4.0,4.0,4.0,3.0,4.0,5.0,8.0,5.0,2.0
100,10.0,3.0,0.0,3.0,5.0,17.0,46.0,68.0,106.0,66.0,...,68.0,69.0,69.0,55.0,58.0,61.0,67.0,52.0,56.0,25.0
101,26.0,10.0,9.0,3.0,20.0,49.0,124.0,245.0,394.0,247.0,...,286.0,316.0,332.0,269.0,239.0,246.0,239.0,209.0,179.0,104.0
118,21.0,11.0,9.0,3.0,1.0,9.0,14.0,61.0,75.0,54.0,...,249.0,279.0,245.0,245.0,287.0,273.0,278.0,281.0,245.0,115.0


In [4]:
# Create a station info dataframe
df_2022 = pd.read_pickle('../data_raw/MTA_Subway_Hourly_Ridership_2022.pkl')
df_2023 = pd.read_pickle('../data_raw/MTA_Subway_Hourly_Ridership_2023.pkl')
df_2022.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,2022-02-17 22:00:00,subway,59,9 Av (D),Brooklyn,omny,OMNY - Full Fare,4,0,40.646294,-73.99432,POINT (-73.99432 40.646294)
1,2022-03-10 23:00:00,subway,404,28 St (6),Manhattan,omny,OMNY - Other,1,0,40.743070,-73.98426,POINT (-73.98426 40.74307)
2,2022-10-05 21:00:00,subway,272,"36 St (M,R)",Queens,omny,OMNY - Full Fare,14,0,40.752040,-73.92878,POINT (-73.92878 40.75204)
3,2022-10-05 22:00:00,subway,420,"219 St (2,5)",Bronx,omny,OMNY - Full Fare,5,0,40.883896,-73.86263,POINT (-73.86263 40.883896)
4,2022-10-05 10:00:00,subway,218,"167 St (B,D)",Bronx,omny,OMNY - Full Fare,78,6,40.833770,-73.91844,POINT (-73.91844 40.83377)


In [44]:
ridership_data_all_years = pd.concat([df_2022, df_2023], axis=0)

In [45]:
# Drop unnessessary columns
columns_to_drop = ['transit_timestamp','transit_mode','payment_method','fare_class_category','ridership','transfers']
ridership_data_all_years.drop(columns_to_drop, axis=1, inplace=True)

In [46]:
ridership_data_all_years.head()

,station_complex_id,station_complex,borough,latitude,longitude,Georeference
0,59,9 Av (D),Brooklyn,40.646294,-73.99432,POINT (-73.99432 40.646294)
1,404,28 St (6),Manhattan,40.743070,-73.98426,POINT (-73.98426 40.74307)
2,272,"36 St (M,R)",Queens,40.752040,-73.92878,POINT (-73.92878 40.75204)
3,420,"219 St (2,5)",Bronx,40.883896,-73.86263,POINT (-73.86263 40.883896)
4,218,"167 St (B,D)",Bronx,40.833770,-73.91844,POINT (-73.91844 40.83377)


In [110]:
def get_attraction_names(ridership_data):
    """
    Add a column of nearby attraction names for each station.
    Returns a DataFrame with all original columns plus nearby attractions.
    
    Parameters:
    ridership_data: DataFrame with station_complex_id, station_complex, borough, latitude, longitude
    """
    # Get unique stations with their info
    unique_stations = ridership_data.groupby('station_complex_id').first().reset_index()
    
    total_stations = len(unique_stations)
    print(f"Processing {total_stations} stations")
    
    # Add empty column for attractions
    unique_stations['nearby_attractions'] = ''
    
    # Process stations
    for idx, row in unique_stations.iterrows():
        try:
            # Construct URL for single station
            url = (
                "https://api.content.tripadvisor.com/api/v1/location/nearby_search"
                f"?latLong={row['latitude']}%2C{row['longitude']}"
                "&key=62E7CCFA411C4FF8BDDE0FB6BEE2C2F6"
                "&category=attractions&radius=.5&radiusUnit=mi&language=en"
            )
            
            response = requests.get(url, headers={"accept": "application/json"})
            response.raise_for_status()
            
            # Extract attraction names from response
            attractions = response.json().get('data', [])
            attraction_names = [attr['name'] for attr in attractions if 'name' in attr]
            
            # Store result
            unique_stations.at[idx, 'nearby_attractions'] = ', '.join(attraction_names)
            
            # Progress update
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{total_stations} stations ({(idx + 1)/total_stations*100:.1f}%)")
                
        except Exception as e:
            print(f"Error processing station {row['station_complex_id']}: {e}")
    
    # Drop Georeference column if not needed
    if 'Georeference' in unique_stations.columns:
        unique_stations = unique_stations.drop('Georeference', axis=1)
    
    return unique_stations

In [111]:
stations_full = get_attraction_names(ridership_data_all_years)
stations_full.head()

Processing 854 stations
Processed 10/854 stations (1.2%)
Processed 20/854 stations (2.3%)
Processed 30/854 stations (3.5%)
Error processing station 45: 429 Client Error:  for url: https://api.content.tripadvisor.com/api/v1/location/nearby_search?latLong=40.64403%2C-73.96449&key=62E7CCFA411C4FF8BDDE0FB6BEE2C2F6&category=attractions&radius=.5&radiusUnit=mi&language=en
Processed 40/854 stations (4.7%)
Processed 50/854 stations (5.9%)
Processed 60/854 stations (7.0%)
Processed 70/854 stations (8.2%)
Processed 80/854 stations (9.4%)
Processed 90/854 stations (10.5%)
Processed 100/854 stations (11.7%)
Processed 110/854 stations (12.9%)
Processed 120/854 stations (14.1%)
Processed 130/854 stations (15.2%)
Processed 140/854 stations (16.4%)
Processed 150/854 stations (17.6%)
Processed 160/854 stations (18.7%)
Processed 170/854 stations (19.9%)
Processed 180/854 stations (21.1%)
Processed 190/854 stations (22.2%)
Processed 200/854 stations (23.4%)
Processed 210/854 stations (24.6%)
Processed 22

,station_complex_id,station_complex,borough,latitude,longitude,nearby_attractions
0,1,"Astoria-Ditmars Blvd (N,W)",Queens,40.775036,-73.91203,"Astoria Internet Cafe, Silver Age Comics, Spa ..."
1,2,"Astoria Blvd (N,W)",Queens,40.770260,-73.91785,"Bana Spa, Deja Vu New York, Omorphia Spa NYC, ..."
2,3,"30 Av (N,W)",Queens,40.766780,-73.92148,"DD, Diamondy Doux: Stephen, Art For After Hour..."
3,4,"Broadway (N,W)",Queens,40.761820,-73.92551,"Break Bar and Billiards, Diamond Works Medspa,..."
4,5,"36 Av (N,W)",Queens,40.756805,-73.92957,"Tantra Lounge, Bld Spa New York, Snowdonia, cu..."


In [112]:
# Convert to a csv file
stations_full.to_csv('station_data.csv', index=False)